In [61]:
import torch
import transformers
from transformers import LlamaTokenizer, LlamaForCausalLM

# Adding Special Tokens to tokenizer

In [62]:
tokenizer = LlamaTokenizer.from_pretrained("maritaca-ai/sabia-7b")

In [63]:
inst = "</INST>"
inst_id = tokenizer(inst, return_tensors="pt")
inst_id

{'input_ids': tensor([[    1,  1533, 25580, 29958]]), 'attention_mask': tensor([[1, 1, 1, 1]])}

In [64]:
special_tokens_dict = {"additional_special_tokens": ["<INST>", "</INST>"]}

tokenizer.add_special_tokens(special_tokens_dict)

2

In [65]:
tokenizer.all_special_tokens

['<s>', '</s>', '<unk>', '<INST>', '</INST>']

In [66]:
test = "<INST>Eu quero que você me gere uma música brasileira</INST>"
input_ids = tokenizer(test, return_tensors="pt")
input_ids

{'input_ids': tensor([[    1, 32000, 16430,   439,  1489,   712,  7931, 30037,   592,   330,
           406,  3672, 15433, 22905,  3055, 32001]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [67]:
tokenizer.decode(list(input_ids['input_ids'][0]), skip_special_tokens=True)

'Eu quero que você me gere uma música brasileira'

In [ ]:
import datasets

dataset = datasets.load_dataset("dominguesm/Canarim-Instruct-PTBR-Dataset")